In [65]:
import requests
import json
import pickle
import csv

# Fetch metadata from IPFS

In [7]:
ON1_MAX = 7777

In [10]:
on1_force_traits_uri = 'https://ipfs.io/ipfs/QmXgSuLPGuxxRuAana7JdoWmaS25oAcXv3x2pYMN9kVfg3/{}'

In [11]:
on1_traits = {}

In [12]:
for i in range(1, ON1_MAX+1):
    traits = requests.get(on1_force_traits_uri.format(i)).json()
    on1_traits[i] = traits

In [13]:
len(on1_traits)

7777

In [17]:
# Save metadata into pickle file so we don't have to make all those requests to IPFS again
with open('on1_force_traits_raw.pickle', 'wb') as handle:
    pickle.dump(on1_traits, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Extract traits

In [26]:
traits = []

In [33]:
for i in range(1, ON1_MAX+1):
    attributes = on1_traits[i]['attributes']
    for trait in attributes:
        traits.append(trait['trait_type'])
traits = sorted(list(set(traits)))

In [34]:
traits

['Background',
 'Body',
 'Domain',
 'Extra',
 'Eyes',
 'Face',
 'Hair',
 'Head',
 'Helmet',
 'Mouth',
 'Spirit',
 'Strength',
 'Style',
 'Type',
 'Wear']

# Transform into CSV

In [40]:
on1_traits[1]

{'name': '0N1 #0001',
 'attributes': [{'trait_type': 'Type', 'value': 'Y0K-A1'},
  {'trait_type': 'Background', 'value': 'Jasper'},
  {'trait_type': 'Body', 'value': 'Jasper'},
  {'trait_type': 'Eyes', 'value': 'Eye Shadow (Jasper)'},
  {'trait_type': 'Mouth', 'value': 'Frown'},
  {'trait_type': 'Wear', 'value': 'Track Jacket (Obsidian)'},
  {'trait_type': 'Hair', 'value': 'Bob (Obsidian)'},
  {'trait_type': 'Face', 'value': 'PR Mask (Citrine)'},
  {'trait_type': 'Head', 'value': 'Void'},
  {'trait_type': 'Extra', 'value': 'Void'},
  {'trait_type': 'Style', 'value': 4, 'max_value': 10},
  {'trait_type': 'Strength', 'value': 5, 'max_value': 10},
  {'trait_type': 'Spirit', 'value': 4, 'max_value': 10}],
 'image': 'ipfs://QmcoavNZq2jyZGe2Zi4nanQqzU9hRPxunHAo8pgYZ5fSep/0001.png'}

In [41]:
# Preprocess the attributes into 1 diciontary so it is easier to work with
for i in range(1, ON1_MAX+1):
    attributes = on1_traits[i]['attributes']
    attribute_dict = {}
    for attribute in attributes:
        attribute_dict[attribute['trait_type']] = attribute['value']
    on1_traits[i]['attributes'] = attribute_dict       

In [69]:
header = [
 'Token Id',
 'Background',
 'Body',
 'Domain',
 'Extra',
 'Eyes',
 'Face',
 'Hair',
 'Head',
 'Helmet',
 'Mouth',
 'Spirit',
 'Strength',
 'Style',
 'Type',
 'Wear',
 'Image Url']

In [72]:
with open('on1_force_traits.csv', 'w', newline='') as outcsv:
    writer = csv.writer(outcsv)
    writer.writerow(header)

    for i in range(1, ON1_MAX+1):
        attributes = on1_traits[i]['attributes']
        row = [i]
        for trait in traits:
            if trait in attributes:
                row.append(attributes[trait])
            else:
                row.append('')
        row.append('https://ipfs.io/ipfs/'+ on1_traits[i]['image'][7:])
        writer.writerow(row)

# Read CSV and generate SQL template

In [91]:
with open('on1_force_traits.csv', newline='') as csvfile:
    r = csv.reader(csvfile, delimiter=',')
    with open('on1_force_traits_sql.txt', 'w') as f:
        for i,row in enumerate(r):
            if(i == 0):
                #Skip header
                continue
            sql_row = '('
            for j,trait in enumerate(row):
                if(j in (0, 11, 12, 13)):
                    sql_row = sql_row + trait + ', '
                else:
                    sql_row = sql_row + '\'{}\''.format(trait) + ', '
            sql_row = sql_row[:-2] + '),'
#             print(sql_row + '\n')
            f.write(sql_row)
            f.write('\n')